7

In [26]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torchvision
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import copy
import tqdm

from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
(train_test_split,
GridSearchCV)
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import sklearn

In [5]:
# Make use of a GPU or MPS (Apple) if one is available.
device = (
    "mps"
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [6]:
# Early Stopping to prevent overfitting

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

In [7]:
df = load_data("Default").dropna()
df

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [8]:
df['student']= df['student'].map({'Yes': 1, 'No' : 0})
df

,default,student,balance,income
0,No,0,729.526495,44361.625074
1,No,1,817.180407,12106.134700
2,No,0,1073.549164,31767.138947
3,No,0,529.250605,35704.493935
4,No,0,785.655883,38463.495879
...,...,...,...,...
9995,No,0,711.555020,52992.378914
9996,No,0,757.962918,19660.721768
9997,No,0,845.411989,58636.156984
9998,No,0,1569.009053,36669.112365


In [9]:
le = preprocessing.LabelEncoder()

x = df[["student", "balance", "income"]].values
y = le.fit_transform(df["default"])
default = le.classes_

In [13]:
# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

 # Numpy to Torch Tensor
x_train = torch.tensor(x_train, device=device, dtype=torch.float32)
y_train = torch.tensor(y_train, device=device, dtype=torch.long)

x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
y_test = torch.tensor(y_test, device=device, dtype=torch.long)

# Create datasets
BATCH_SIZE = 16

dataset_train = TensorDataset(x_train, y_train)
dataloader_train = DataLoader(
    dataset_train, batch_size=BATCH_SIZE, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)

# Create model using nn.Sequential, i used 50 layers instead of the recommended 10
model = nn.Sequential(
    nn.Linear(x_train.shape[1], 50), 
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(50, 25), 
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(25, len(default)),
    nn.LogSoftmax(dim=1),
)



loss_fn = nn.CrossEntropyLoss()  # cross entropy loss

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
es = EarlyStopping()

epoch = 0
done = False
while epoch < 1000 and not done:
    epoch += 1
    steps = list(enumerate(dataloader_train))
    pbar = tqdm.tqdm(steps)
    model.train()
    for i, (x_batch, y_batch) in pbar:
        y_batch_pred = model(x_batch.to(device))
        loss = loss_fn(y_batch_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), (i + 1) * len(x_batch)
        if i == len(steps) - 1:
            model.eval()
            pred = model(x_test)
            vloss = loss_fn(pred, y_test)
            if es(model, vloss):
                done = True
            pbar.set_description(
                f"Epoch: {epoch}, tloss: {loss}, vloss: {vloss:>7f}, {es.status}"
            )
        else:
            pbar.set_description(f"Epoch: {epoch}, tloss {loss:}")

Epoch: 1, tloss: 0.03357246518135071, vloss: 0.082374, : 100%|██████████| 469/469 [00:00<00:00, 597.09it/s]
Epoch: 2, tloss: 0.07549075037240982, vloss: 0.080851, Improvement found, counter reset to 0: 100%|██████████| 469/469 [00:00<00:00, 657.19it/s]
Epoch: 3, tloss: 0.02112645097076893, vloss: 0.082565, No improvement in the last 1 epochs: 100%|██████████| 469/469 [00:00<00:00, 634.07it/s]
Epoch: 4, tloss: 0.010429519228637218, vloss: 0.079969, Improvement found, counter reset to 0: 100%|██████████| 469/469 [00:00<00:00, 631.50it/s]
Epoch: 5, tloss: 0.009180329740047455, vloss: 0.079980, No improvement in the last 1 epochs: 100%|██████████| 469/469 [00:00<00:00, 634.06it/s]
Epoch: 6, tloss: 0.01099956501275301, vloss: 0.082594, No improvement in the last 2 epochs: 100%|██████████| 469/469 [00:00<00:00, 638.38it/s]
Epoch: 7, tloss: 0.07320550829172134, vloss: 0.081538, No improvement in the last 3 epochs: 100%|██████████| 469/469 [00:00<00:00, 622.28it/s]
Epoch: 8, tloss: 0.015343118

In [14]:
pred = model(x_test)
vloss = loss_fn(pred, y_test)
print(f"Loss = {vloss}")

Loss = 0.07996878027915955


In [15]:
from sklearn.metrics import accuracy_score

pred = model(x_test)
_, predict_classes = torch.max(pred, 1)
correct = accuracy_score(y_test.cpu(), predict_classes.cpu())
print(f"Accuracy: {correct}")

Accuracy: 0.9732


The Neural Network managed a 97% accuracy, albeit this is a very simple dataset.

In [19]:
df = load_data('Default').dropna()
df

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [20]:
df['student']= df['student'].map({'Yes': 1, 'No' : 0})
df['default']= df['default'].map({'Yes': 1, 'No' : 0})
df

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879
...,...,...,...,...
9995,0,0,711.555020,52992.378914
9996,0,0,757.962918,19660.721768
9997,0,0,845.411989,58636.156984
9998,0,0,1569.009053,36669.112365


In [22]:
scaler = preprocessing.StandardScaler()
model = MS(df.columns.drop('default'), intercept= False)
d = model.fit_transform(df)
x = np.asarray(d)
y = df['default'].values

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [28]:
logit = sklearn.linear_model.LogisticRegression(random_state= 42)
logit.fit(x_train,y_train)

pred = logit.predict(x_test)

correct = accuracy_score(y_test, pred )
print(f"Accuracy: {correct}")

Accuracy: 0.9672


The Logistic Regression only performed marginally worse than the Neural Classifier, with a 96% accuracy